In [ ]:
import torch
import segmentation_models_pytorch as smp
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader

In [ ]:
# CONFIG
IMG_SIZE = 256
NUM_CLASSES = 5
CLASS_NAMES = ["LULC", "River", "Road", "Settlement", "Soil"]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

In [ ]:
import glob

class MapDataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, lbl_dir):
        self.img_paths = sorted(glob.glob(img_dir + "/*.jpg"))
        self.lbl_paths = sorted(glob.glob(lbl_dir + "/*.txt"))

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        img = img.resize((IMG_SIZE, IMG_SIZE))
        img = np.array(img) / 255.0
        img = np.transpose(img, (2, 0, 1)).astype(np.float32)

        label = int(open(self.lbl_paths[idx]).read().strip())
        return torch.tensor(img), torch.tensor(label)

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/RemoteSensingProject/Dataset_Final"

val_ds = MapDataset(
    f"{DATASET_DIR}/val/images",
    f"{DATASET_DIR}/val/labels"
)

val_dl = DataLoader(val_ds, batch_size=16, shuffle=False)
print("Validation samples:", len(val_ds))

In [ ]:
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights=None,
    in_channels=3,
    classes=NUM_CLASSES
).to(device)

MODEL_PATH = "models/unet_resnet34.pth"
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.eval()

print("✅ Model loaded successfully")

In [ ]:
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    img = img.resize((IMG_SIZE, IMG_SIZE))
    arr = np.array(img) / 255.0
    tensor = torch.tensor(
        arr.transpose(2, 0, 1),
        dtype=torch.float32
    ).unsqueeze(0).to(device)

    with torch.no_grad():
        out = model(tensor)
        out = out.mean(dim=(2, 3))
        probs = torch.softmax(out, dim=1)[0].cpu().numpy()

    cls = np.argmax(probs)
    conf = probs[cls] * 100

    print(f"Predicted Class : {CLASS_NAMES[cls]}")
    print(f"Confidence      : {conf:.2f}%")

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for imgs, lbls in val_dl:
        imgs, lbls = imgs.to(device), lbls.to(device)
        out = model(imgs)
        out = out.mean(dim=(2, 3))
        preds = torch.argmax(out, dim=1)

        correct += (preds == lbls).sum().item()
        total += lbls.size(0)

accuracy = correct / total
print(f"📌 Validation Accuracy: {accuracy*100:.2f}%")